<a href="https://colab.research.google.com/github/AshishGusain17/models/blob/master/ipynb/local/videos-tf2-obj-det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --user Cython
!pip install --user contextlib2
!pip install --user pillow
!pip install --user lxml
!pip install --user jupyter
!pip install --user matplotlib

In [5]:
%cd tf-obj-detection/research
!python setup.py build
!python setup.py install
# %cd ../

C:\Users\hp\Desktop\tf-obj-detection\research
running build
running build_py
copying object_detection\protos\anchor_generator_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\argmax_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\bipartite_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_predictor_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\calibration_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\eval_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\flexible_grid_anchor_generator_pb2.py -> build\lib\object_detection\protos
copying

zip_safe flag not set; analyzing archive contents...
object_detection.core.__pycache__.preprocessor.cpython-37: module MAY be using inspect.stack
object_detection.utils.__pycache__.autoaugment_utils.cpython-37: module MAY be using inspect.stack


In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [0]:
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/faster_rcnn_resnet101_coco_2018_01_28' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28' '/content/models/research/object_detection'

!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/training' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/images' '/content/models/research/object_detection'

In [0]:

%cd /content/models/research
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
%cd object_detection/legacy
%cd ..
%cd ..
%cd ..
%cd ..
%cd ..

/content/models/research
env: PYTHONPATH=/content/models/research:/content/models/research/slim
/content/models/research/object_detection/legacy
/content/models/research/object_detection
/content/models/research
/content/models
/content
/


In [7]:
%ls
# %cd research


 Volume in drive C is Windows
 Volume Serial Number is 68D3-0A2F

 Directory of C:\Users\hp\Desktop\tf-obj-detection\research

23-04-2020  13:47    <DIR>          .
23-04-2020  13:47    <DIR>          ..
22-04-2020  01:18    <DIR>          a3c_blogpost
22-04-2020  01:18    <DIR>          adv_imagenet_models
22-04-2020  01:18    <DIR>          adversarial_crypto
22-04-2020  01:18    <DIR>          adversarial_logit_pairing
22-04-2020  01:18    <DIR>          adversarial_text
22-04-2020  01:18    <DIR>          attention_ocr
22-04-2020  01:18    <DIR>          audioset
22-04-2020  01:18    <DIR>          autoaugment
22-04-2020  01:18    <DIR>          autoencoder
22-04-2020  01:18    <DIR>          brain_coder
22-04-2020  01:31    <DIR>          build
22-04-2020  01:18    <DIR>          cognitive_mapping_and_planning
22-04-2020  01:18    <DIR>          cognitive_planning
22-04-2020  01:18    <DIR>          compression
22-04-2020  01:18    <DIR>          cvt_text
22-04-2020  01:18    <DIR

In [8]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import os
import pathlib
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [9]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [10]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [26]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [27]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[]

In [28]:
model_dir =  "object_detection/faster_rcnn_inception_v2_coco_2018_01_28/saved_model"
model = tf.saved_model.load(str(model_dir))
detection_model = model.signatures['serving_default']
# faster_rcnn_resnet101_coco_2018_01_28
# mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28
# http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [29]:
print(detection_model.inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [30]:
detection_model.output_dtypes

{'detection_boxes': tf.float32,
 'detection_scores': tf.float32,
 'detection_classes': tf.float32,
 'num_detections': tf.float32}

In [31]:
detection_model.output_shapes

{'detection_boxes': TensorShape([None, 100, 4]),
 'detection_scores': TensorShape([None, 100]),
 'detection_classes': TensorShape([None, 100]),
 'num_detections': TensorShape([None])}

In [32]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [33]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  # image_np = np.array(Image.open(image_path))
  image_np = np.array(image_path)
  print(image_np.shape)

  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  # display(Image.fromarray(image_np))
  return image_np

In [0]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

In [85]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time
from sklearn.metrics import pairwise
from imutils.video import FPS


# cap=cv2.VideoCapture(0)
cap=cv2.VideoCapture("../../GitHub/yolo_object_trace/videos/a.mp4")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out1 = cv2.VideoWriter('i.avi', fourcc, 3.0, (int(cap.get(3)),int(cap.get(4))))
fps = FPS().start()

time.sleep(2.0)

ctt = 0
while True:
    (grabbed, frame) = cap.read()
    print('frame',frame.shape)

    # resize the frame, blur it, and convert it to the HSV color space
    # frame = imutils.resize(frame, width=600)
    # print('frame_resize',frame.shape)
    print(ctt)
    ctt = ctt + 1
    frame=show_inference(detection_model, frame)
    if ctt>1670:
        break

    cv2.imshow("version", frame)
    out1.write(frame)
    key=cv2.waitKey(100)
    if key & 0xFF == ord("q"):
        break

    # update the FPS counter
    fps.update()

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
cap.release()
out1.release()
cv2.destroyAllWindows() 

frame (720, 1280, 3)
0
(720, 1280, 3)
frame (720, 1280, 3)
1
(720, 1280, 3)
frame (720, 1280, 3)
2
(720, 1280, 3)
frame (720, 1280, 3)
3
(720, 1280, 3)
frame (720, 1280, 3)
4
(720, 1280, 3)
frame (720, 1280, 3)
5
(720, 1280, 3)
frame (720, 1280, 3)
6
(720, 1280, 3)
frame (720, 1280, 3)
7
(720, 1280, 3)
frame (720, 1280, 3)
8
(720, 1280, 3)
frame (720, 1280, 3)
9
(720, 1280, 3)
frame (720, 1280, 3)
10
(720, 1280, 3)
frame (720, 1280, 3)
11
(720, 1280, 3)
frame (720, 1280, 3)
12
(720, 1280, 3)
frame (720, 1280, 3)
13
(720, 1280, 3)
frame (720, 1280, 3)
14
(720, 1280, 3)
frame (720, 1280, 3)
15
(720, 1280, 3)
frame (720, 1280, 3)
16
(720, 1280, 3)
frame (720, 1280, 3)
17
(720, 1280, 3)
frame (720, 1280, 3)
18
(720, 1280, 3)
frame (720, 1280, 3)
19
(720, 1280, 3)
frame (720, 1280, 3)
20
(720, 1280, 3)
frame (720, 1280, 3)
21
(720, 1280, 3)
frame (720, 1280, 3)
22
(720, 1280, 3)
frame (720, 1280, 3)
23
(720, 1280, 3)
frame (720, 1280, 3)
24
(720, 1280, 3)
frame (720, 1280, 3)
25
(720, 1280,

In [81]:
cd research

C:\Users\hp\Desktop\tf-obj-detection\research


In [79]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 68D3-0A2F

 Directory of C:\Users\hp\Desktop\tf-obj-detection

23-04-2020  23:52    <DIR>          .
23-04-2020  23:52    <DIR>          ..
22-04-2020  01:18    <DIR>          .github
22-04-2020  01:18             1,398 .gitignore
22-04-2020  01:18                 0 .gitmodules
05-03-2020  11:25       134,056,826 a.mp4
22-04-2020  01:18               347 AUTHORS
22-04-2020  01:18             2,525 CODEOWNERS
22-04-2020  01:18             1,476 CONTRIBUTING.md
23-04-2020  23:52    <DIR>          ipynb
22-04-2020  01:18             2,093 ISSUE_TEMPLATE.md
22-04-2020  01:18             1,146 ISSUES.md
22-04-2020  01:18            11,608 LICENSE
22-04-2020  01:18    <DIR>          official
22-04-2020  01:18             1,279 README.md
23-04-2020  23:17    <DIR>          research
22-04-2020  01:18                 0 WORKSPACE
              11 File(s)    134,078,698 bytes
               6 Dir(s)  719,076,929,536 bytes free
